In [46]:
import pandas as pd
import numpy as np
import os
import random
import progressbar
import pickle
import matplotlib.pyplot as plt

from sklearn.linear_model import (GammaRegressor, PoissonRegressor,
                                  TweedieRegressor)
from sklearn.model_selection import cross_validate
!pip install -U imbalanced-learn
from imblearn.over_sampling import (ADASYN, BorderlineSMOTE, KMeansSMOTE,
                                    RandomOverSampler, SMOTE, SMOTEN, SMOTENC,
                                    SVMSMOTE)
from imblearn.under_sampling import (AllKNN, ClusterCentroids,
                                     CondensedNearestNeighbour,
                                     EditedNearestNeighbours,
                                     InstanceHardnessThreshold,
                                     NearMiss, NeighbourhoodCleaningRule,
                                     OneSidedSelection, RandomUnderSampler,
                                     RepeatedEditedNearestNeighbours,
                                     TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek

import seaborn as sb

from google.colab import drive
drive.mount('/content/gdrive')

from sklearn.model_selection import cross_validate
!pip install -U imbalanced-learn
from imblearn.over_sampling import (ADASYN, BorderlineSMOTE, KMeansSMOTE,
                                    RandomOverSampler, SMOTE, SMOTEN, SMOTENC,
                                    SVMSMOTE)
from imblearn.under_sampling import (AllKNN, ClusterCentroids,
                                     CondensedNearestNeighbour,
                                     EditedNearestNeighbours,
                                     InstanceHardnessThreshold,
                                     NearMiss, NeighbourhoodCleaningRule,
                                     OneSidedSelection, RandomUnderSampler,
                                     RepeatedEditedNearestNeighbours,
                                     TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek

import seaborn as sb

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Total number of data

In [33]:
data_per_csv = 512
def total():
  total_files = 0
  for files in os.listdir('gdrive/My Drive/Summer Research/HRV/Outlier Free/All/'):
    total_files += 1
  return total_files

Load data

In [34]:
def loadHRVData(c):
  hrv_and_labels = list()
  if c == 'wt':
    with open('gdrive/My Drive/Summer Research/Variables/wt_pseudoimage_hrv_and_labels.pkl', 'rb') as file:
      #load data from file
      hrv_and_labels = pickle.load(file)
  elif c == 'wt denoised':
    with open('gdrive/My Drive/Summer Research/Variables/wt_denoised_pseudoimage_hrv_and_labels.pkl', 'rb') as file:
      #load data from file
      hrv_and_labels = pickle.load(file)
  elif c == 'normal':
    size = (163, 223, 4)
    with open('gdrive/My Drive/Summer Research/Variables/normal_hrv_and_labels.pkl', 'rb') as file:
      #load data from file
      hrv_and_labels = pickle.load(file)
  elif c == 'array':
    with open('gdrive/My Drive/Summer Research/Variables/array_hrv_and_labels.pkl', 'rb') as file:
      #load data from file
      hrv_and_labels = pickle.load(file)
  elif c == 'wt a1d1d2d3 coords':
    with open('gdrive/My Drive/Summer Research/Variables/wt_a1d1d2d3_coords_hrv_and_labels.pkl', 'rb') as file:
      #save data to a file
      hrv_and_labels = pickle.load(file)
  elif c == 'wt a1d1d2d3 denoised coords':
    with open('gdrive/My Drive/Summer Research/Variables/wt_a1d1d2d3_denoised_coords_hrv_and_labels.pkl', 'rb') as file:
      #save data to a file
      hrv_and_labels = pickle.load(file)
  return hrv_and_labels

Confusion matrix (tp+fp+tn+fn set to equal 1)

In [35]:
def calculate_average_confusion_matrix(recall, precision, accuracy, method):
  fp_list = list()
  tp_list = list()
  if method == 'each':
    # tp/(tp+fp)=Precision, tp/(tp+fn)=Recall, (tp+tn)/(tp+fn+fp+tn)=Accuracy
    tp_avg = fp_avg = tn_avg = fn_avg = 0
    for i in range(len(recall)):
      fn = (1 / recall[i]) - 1
      fp = (1 / precision[i]) - 1
      tp = 1
      tn = (1 - accuracy[i] - accuracy[i] * fn - accuracy[i] * fp) / (accuracy[i] - 1)
      fp_list.append(fp/(fp+tn))
      tp_list.append(tp/(tp+fn))

      tp_avg += tp
      fp_avg += fp
      tn_avg += tn
      fn_avg += fn
    sum = fn_avg + fp_avg + tp_avg + tn_avg
    fn_avg, fp_avg, tp_avg, tn_avg = fn_avg/sum, fp_avg/sum, tp_avg/sum, tn_avg/sum
  elif method == 'avg':
    r_avg = recall.sum()/len(recall)
    p_avg = precision.sum()/len(precision)
    a_avg = accuracy.sum()/len(accuracy)

    fn = (1 / r_avg) - 1
    fp = (1 / p_avg) - 1
    tp = 1
    tn = (1 - a_avg - a_avg * fn - a_avg * fp) / (a_avg - 1)

    sum = fn + fp + tp + tn
    fn_avg, fp_avg, tp_avg, tn_avg = fn/sum, fp/sum, tp/sum, tn/sum
    fp_list.append(fp_avg)
    tp_list.append(tp_avg)
  
  tpr = tp/(tp+fn)
  tnr = tn/(tn+fp)
  return (tpr+tnr)/2, np.array([[tp_avg, fp_avg], [fn_avg, tn_avg]]), fp_list, tp_list

Oversampling and undersampling

In [36]:
def resampling(args):
  if args == 'SMOTEENN':
    resampler = SMOTEENN(n_jobs=-1)
  elif args == 'SMOTETomek':
    resampler = SMOTETomek(n_jobs=-1)
  
  return resampler

GLM model

In [47]:
def GLMModel(X, y):
  scores = []
  avg_acc = []
  #params = [10]

  model = GammaRegressor()
  #K-fold Cross Validation
  scores = cross_validate(model, X, y, cv=10, scoring=('accuracy', 'neg_root_mean_squared_error', 'precision', 'recall', 'roc_auc'), n_jobs=-1, verbose=0, return_train_score=False)
  return scores

In [38]:
def metrics(scores, resampling_method, data_choice):
  which_metric = 'test'
  acc = scores[which_metric+'_accuracy']
  rmse = scores[which_metric+'_neg_root_mean_squared_error']
  precision = scores[which_metric+'_precision']
  recall = scores[which_metric+'_recall']
  auc = scores[which_metric+'_roc_auc']
  bAcc, matrix, fp, tp = calculate_average_confusion_matrix(recall, precision, acc, 'each')

  dir = 'gdrive/My Drive/Summer Research/Figures/GLM/'
  file_name = resampling_method+'-resampled '+data_choice

  with open(dir+file_name+'.txt', 'w') as file:
    file.write("Accuracy:"+str(acc))
    file.write("\nBalanced accuracy: "+str(bAcc))
    file.write("\nRMSE:"+str(rmse))
    file.write("\nPrecision:"+str(precision))
    file.write("\nRecall:"+str(recall))
    file.write("\nROC AUC:"+str(auc))
  #TODO: generate PR, ROC, PROC graphs

  plt.figure(figsize=(12,8))
  plt.scatter(fp, tp, label="HRV, auc="+str(auc))
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.xlim((0,1))
  plt.ylim((0,1))
  plt.show()

  plt.figure(figsize=(12,8))
  plt.scatter(precision, recall)
  plt.xlabel('Precision')
  plt.ylabel('Recall')
  plt.title('Precision v. Recall')
  plt.xlim((0,1))
  plt.ylim((0,1))
  plt.show()

  plt.figure(figsize=(12,8))
  g = sb.heatmap(matrix, annot=True, fmt='.2%', cmap='Greens', xticklabels=['Predicted Healthy', 'Predicted Diabetes'], yticklabels=['Healthy', 'Diabetes'])
  g.set_yticklabels(labels=g.get_yticklabels(), va='center')
  g.set_title('Confusion Matrix')
  plt.savefig(dir+file_name)

In [48]:
data_choices = {
    1:'wt',
    2:'wt denoised',
    3:'array',
    4:'wt a1d1d2d3 coords',
    5:'wt a1d1d2d3 denoised coords'
}

widgets = [' [',
      progressbar.Timer(format= 'elapsed time: %(elapsed)s'),
      '] ',
        progressbar.Bar('#'),' (',
        progressbar.ETA(), ') ',
        ]
bar = progressbar.ProgressBar(max_value=10, widgets=widgets).start()
count = 0

for r in ['SMOTETomek', 'SMOTEENN']:
  resampling_method = r
  for i in range(len(data_choices)):
    count += 1
    bar.update(count)
    data_choice = data_choices[1]
    hrv_and_labels = loadHRVData(data_choice)

    random.shuffle(hrv_and_labels)
    X = np.array([item[0] for item in hrv_and_labels]).reshape(total(),-1)
    y = np.array([item[1] for item in hrv_and_labels])

    X_resampled, y_resampled = resampling(resampling_method).fit_resample(X, y)
    scores = GLMModel(X_resampled, y_resampled)
    metrics(scores, resampling_method, data_choice)

 [elapsed time: 0:00:00] |                                  | (ETA:  --:--:--) 

FileNotFoundError: ignored